In [1]:
import pandas as pd
import numpy as np

### Books Rating Dataframe

This file has information about 3M book reviews for 212404 unique book and users who gives these reviews for each book.

In [2]:
df_rating = pd.read_csv("/kaggle/input/amazon-books-reviews/Books_rating.csv")
df_rating.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [3]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


## 1. **Data Cleaning and Preprocessing**
   - *Objective*: Prepare the dataset for analysis by handling missing values, correcting data inconsistencies, and transforming data into a usable format.
   - *Business Value*: Ensures high-quality data for analysis, leading to more accurate and reliable results.
   - *Approach*: Implement data cleaning operations such as filling missing values, removing duplicates, normalizing text data, and encoding categorical variables.


In [4]:
# Check for duplicates
df_rating.duplicated()

0          False
1          False
2          False
3          False
4          False
           ...  
2999995    False
2999996    False
2999997    False
2999998    False
2999999    False
Length: 3000000, dtype: bool

In [5]:
# Count the number of duplicate rows
duplicate_count = df_rating.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 8774


In [6]:
# Remove duplicates
df_rating.drop_duplicates(inplace = True)

duplicate_count = df_rating.duplicated().sum()
print(f"Number of duplicate rows after removing duplicates: {duplicate_count}")
print('There are {} rows and {} columns in the dataframe after removing duplicates.'.format(df_rating.shape[0],df_rating.shape[1]))

Number of duplicate rows after removing duplicates: 0
There are 2991226 rows and 10 columns in the dataframe after removing duplicates.


In [7]:
# Check for missing values
missing_values = df_rating.isnull().sum()
print(missing_values)

Id                          0
Title                     208
Price                 2510854
User_id                558559
profileName            558677
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64


In [8]:
# List of columns to drop
columns_to_drop = ['Id', 'User_id', 'profileName', 'review/time']

# Drop the unnecessary columns with null values
df_rating = df_rating.drop(columns=columns_to_drop)

df_rating.columns

Index(['Title', 'Price', 'review/helpfulness', 'review/score',
       'review/summary', 'review/text'],
      dtype='object')

In [9]:
# Remove rows where the null value count is insignificant
df_rating.dropna(subset=['Title', 'review/summary', 'review/text'], inplace = True)

missing_values = df_rating.isnull().sum()
print("Missing values:\n", missing_values)

Missing values:
 Title                       0
Price                 2510275
review/helpfulness          0
review/score                0
review/summary              0
review/text                 0
dtype: int64


In [11]:
print('There are {} rows and {} columns in the dataframe after removing null values.'.format(df_rating.shape[0],df_rating.shape[1]))

There are 2990603 rows and 6 columns in the dataframe after removing null values.


In [12]:
# Lowecasing & Removing punctuations and stopwords & Lemmatization

#Libraries for formattting and handling text 
import string
import re

#Library for nltk
import nltk 
import nltk.corpus
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Lowercase, punctuations and stopwords
nltk.download('stopwords')

# Get the stopwords list
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()  # Converts text to lowercase
    text = re.sub('\d+', '', text)  # Removes numbers
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Removes punctuations
    
    # Split text into words
    words = text.split()
    
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    
    # Join words back into a single string
    text = ' '.join(words)
    
    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Apply clean_text function to the 'review/summary' column
df_rating['Cleaned review/summary'] = df_rating['review/summary'].apply(clean_text)
df_rating[['review/summary', 'Cleaned review/summary']].head()

,review/summary,Cleaned review/summary
0,Nice collection of Julie Strain images,nice collection julie strain images
1,Really Enjoyed It,really enjoyed
2,Essential for every personal and Public Library,essential every personal public library
3,Phlip Nel gives silly Seuss a serious treatment,phlip nel gives silly seuss serious treatment
4,Good academic overview,good academic overview


In [20]:
# Apply clean_text function to the 'review/text' column
df_rating['Cleaned review/text'] = df_rating['review/text'].apply(clean_text)
df_rating[['review/text','Cleaned review/text']].head()

,review/text,Cleaned review/text
0,This is only for Julie Strain fans. It's a col...,julie strain fans collection photos pages wort...
1,I don't care much for Dr. Seuss but after read...,dont care much dr seuss reading philip nels bo...
2,"If people become the books they read and if ""t...",people become books read child father man dr s...
3,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",theodore seuss geisel aka quotdr seussquot one...
4,Philip Nel - Dr. Seuss: American IconThis is b...,philip nel dr seuss american iconthis basicall...


In [40]:
# Lemmatization

!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.5 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
import pandas as pd

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Define the function to lemmatize a batch of texts
def lemmatize_text_batch(texts):
    docs = list(nlp.pipe(texts, batch_size=1000))
    lemmatized_texts = [' '.join([token.lemma_ for token in doc if not token.is_punct]) for doc in docs]
    return lemmatized_texts

# Define the function to process in batches and update the DataFrame
def process_in_batches_v2_multiple_columns(df, input_columns, output_columns, batch_size):
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        batch = df.loc[start:end, input_columns]
        
        # Print a sample of the data before lemmatization
        print(f"Processing rows {start} to {end}...")
        for i, col in enumerate(input_columns):
            print(f"Sample before lemmatization from column '{col}':")
            print(batch[col].head(2))  # Print the first 2 entries for brevity
        
        # Process each column separately
        for i, col in enumerate(input_columns):
            lemmatized_batch = lemmatize_text_batch(batch[col])
            df.loc[start:end, output_columns[i]] = lemmatized_batch
        
        # Print a sample of the data after lemmatization
        for i, col in enumerate(output_columns):
            print(f"Sample after lemmatization from column '{col}':")
            print(df.loc[start:end, col].head(2))  # Print the first 2 entries for brevity

    return df

# Apply the function to your DataFrame
batch_size = 500000  # Adjust as needed
input_columns = ['Cleaned review/summary', 'Cleaned review/text']
output_columns = ['Lemmatized review/summary', 'Lemmatized review/text']
df_rating = process_in_batches_v2_multiple_columns(df_rating, input_columns, output_columns, batch_size)

Processing rows 0 to 10000...
Sample before lemmatization from column 'Cleaned review/summary':
0    nice collection julie strain images
1                         really enjoyed
Name: Cleaned review/summary, dtype: object
Sample before lemmatization from column 'Cleaned review/text':
0    julie strain fans collection photos pages wort...
1    dont care much dr seuss reading philip nels bo...
Name: Cleaned review/text, dtype: object
Sample after lemmatization from column 'Lemmatized review/summary':
0    nice collection julie strain image
1                          really enjoy
Name: Lemmatized review/summary, dtype: object
Sample after lemmatization from column 'Lemmatized review/text':
0    julie strain fans collection photo page worth ...
1    do not care much dr seuss reading philip nel b...
Name: Lemmatized review/text, dtype: object
Processing rows 10000 to 20000...
Sample before lemmatization from column 'Cleaned review/summary':
10000    published long time ago still relevant to

In [ ]:
df_rating[['Cleaned review/summary', 'Lemmatized review/summary', 'Cleaned review/text', 'Lemmatized review/text']].head()

In [ ]:
df_rating.columns

In [ ]:
# List of columns to drop
columns_to_drop = ['review/helpfulness']

# Drop the unnecessary columns with null values
df_rating = df_rating.drop(columns=columns_to_drop)

df_rating.columns

In [ ]:
# Columns to include
columns_to_include = ['Cleaned Title', 'Lemmatized review/summary', 'Lemmatized review/text']
df_rating_subset = df_rating[columns_to_include]

# Save the subset to a CSV file
df_rating_subset.to_csv('Books_rating_cleaned.csv', index=False)